# Simulation of the Nominal Process

Import PyRolL modules.

In [9]:
import pyroll.basic as pr

Import the process data.

In [10]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE
from weiner_variation.sim.config import FIELDS

Import further libraries.

In [11]:
import pandas as pd
from copy import deepcopy

Parameters for notebook using papermill.

In [12]:
OUTPUT_FILENAME = "sim_nominal_results.csv"

Create a deep copy of the pass sequence an run simulation routine.

In [13]:
sequence = deepcopy(PASS_SEQUENCE)
sequence.solve(IN_PROFILE)       

Profile(
    classifiers={'generic_elongation', 'round'},
    cross_section=Polygon(
        area=5.1043645930000824e-05,
        height=0.008,
        perimeter=0.02540371633395158,
        width=0.008052340169554842,
        ),
    density=7500.0,
    freiberg_flow_stress_coefficients=FreibergFlowStressCoefficients(a=2731390000.0, m1=-0.00268, m2=0.31076, m3=0, m4=-0.00056, m5=0.00046, m6=0, m7=-0.98375, m8=0.000139, m9=0, baseStrain=0.1, baseStrainRate=0.1),
    grain_size=0.0002,
    length=0.0,
    material='C45',
    recrystallized_fraction=0,
    specific_heat_capacity=690,
    strain=0.41232106122177925,
    t=63.76635093954749,
    temperature=1279.5286276154063,
    )

Extract results from pass sequence.

In [14]:
result = { ("draw", "diameter"): IN_PROFILE.diameter, ("draw", "temperature"): IN_PROFILE.temperature } | (
    FIELDS.items()
    |> starmap$((key, extractor) -> (
            sequence.units 
            |> filter$(u -> isinstance(u, pr.RollPass)) 
            |> map$(u -> ((key, u.label), extractor(u))) 
            |> filter$(t -> t[1] is not None)
        )
    )
    |> flatten
    |> dict            
)

Create a dataframe of results.

In [15]:
df = result |> pd.Series |> .T |> .infer_objects()
df

draw                         diameter            0.050000
                             temperature      1423.150000
roll_force                   R1             301382.572364
                             R2             175904.037720
                             R3             318726.081249
                                                ...      
out_recrystallized_fraction  R10                 0.000000
                             F1                  0.000000
                             F2                  0.000000
                             F3                  0.000000
                             F4                  0.000000
Length: 184, dtype: float64

Save dataframe to CSV file.

In [16]:
df.to_csv(OUTPUT_FILENAME)